In [179]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from random import sample
import seaborn as sns

from statsmodels import regression
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.stattools import coint

plt.rcParams["figure.figsize"] = (15,7)

In [180]:
start = "2016-05-01"
end = "2021-01-01"
samples = ["AM","AR","APA","AROC","BKR","BSM","BP","BPMP","COG","WHD","CPE","CVE","CDEV","CHX","LNG","CQP","CHK","CVX","XEC","CLNE","CNX","CRK","COP","CLR","CEQP","CVI","DCP","DKL","DK","DEN","DVN","FANG","DNOW","ET","DRQ","ENBL","ERF","ENLC","EPD","EOG","EQT","ETRN","XOM","GEL","HAL","HP","HES","HEP","HFC","KMI","KOS","LBRT","MMP","MGY","MRO","MPC","MTDR","MPLX","MUR","NESR","NFG","NOV","NBLX","NS","OAS","OXY","OII","OKE","PTEN","PBF","PDCE","PSX","PSXP","PXD","PAA","PAGP","RRC","RTLR","REGI","RES","SOL","SLB","SHLX","OILY","SM","SWN","SUN","TRGP","FTI","TPL","TGS","WMB","RIG","USAC","VLO","VVV","VET","VNOM","WES","WLL","INT"]
data = yf.download(tickers=samples, start=start, end=end, interval = "1d")
prices = data['Adj Close'].dropna(axis=1)


[*********************100%***********************]  101 of 101 completed

2 Failed downloads:
- CHK: Data doesn't exist for startDate = 1462086000, endDate = 1609488000
- DEN: Data doesn't exist for startDate = 1462086000, endDate = 1609488000


In [181]:
oilUS = pd.read_csv('https://raw.githubusercontent.com/LDeng0205/Pairs-Trading-with-Macroeconomic-Data-Analysis/master/oil-data/DCOILWTICO.csv')
oilEU = pd.read_csv('https://raw.githubusercontent.com/LDeng0205/Pairs-Trading-with-Macroeconomic-Data-Analysis/master/oil-data/DCOILBRENTEU.csv')
dgs5 = pd.read_csv('https://raw.githubusercontent.com/LDeng0205/Pairs-Trading-with-Macroeconomic-Data-Analysis/master/oil-data/DGS5.csv')
T5YIE = pd.read_csv('https://raw.githubusercontent.com/LDeng0205/Pairs-Trading-with-Macroeconomic-Data-Analysis/master/oil-data/T5YIE.csv')
unrate = pd.read_csv('https://raw.githubusercontent.com/LDeng0205/Pairs-Trading-with-Macroeconomic-Data-Analysis/master/oil-data/UNRATE.csv')
# oilUS #daily from 2016 april 19
# oilEU #daily from 2016 april 19
# dgs5 #daily from 2016 april 25
# T5YIE #daily from 2016 april 25
# unrate #monthly from 2016 march 1

def dateAdjust(raw, start_date, end_date):
  mask = (raw['DATE'] >= start_date) & (raw['DATE'] < end_date)
  return raw.loc[mask]

def clean(a, ticker):
  a[a == "."] = '0'
  a[ticker] = a[ticker].astype("float64")
  a.DATE = pd.to_datetime(a.DATE)
  clean = a.set_index(["DATE"])
  clean_resampled = clean.resample("1d").interpolate()
  return clean_resampled[ticker]



oilUS = dateAdjust(oilUS, start, end)
oilEU = dateAdjust(oilEU, start, end)
dgs5 = dateAdjust(dgs5, start, end)
T5YIE = dateAdjust(T5YIE, start, end)
unrate = dateAdjust(unrate, start, end)


a1 = clean(oilUS,"DCOILWTICO").to_numpy()
a2 = clean(oilEU,"DCOILBRENTEU").to_numpy()
a3 = clean(dgs5,"DGS5").to_numpy()
a4 = clean(T5YIE,"T5YIE").to_numpy()
const = np.ones_like(a1, dtype = 'float64')

A = np.vstack((a1, a2, a3, a4, const)).T

#A, containing information about macroeconomic data, is the matrix for least squares
#least squares equation A * Wn = Pn
#where p is the stock price time series of the company

print(A)
print(A.shape)

print(len(data))
resampled_data = prices.resample("1d").interpolate().to_numpy()
print(resampled_data.shape)


[[44.75 45.82  1.32  1.58  1.  ]
 [43.65 43.09  1.25  1.54  1.  ]
 [43.77 43.08  1.23  1.52  1.  ]
 ...
 [47.85 50.44  0.37  1.9   1.  ]
 [48.24 50.74  0.37  1.92  1.  ]
 [48.35 51.22  0.36  1.95  1.  ]]
(1705, 5)
1177
(1705, 88)


In [182]:
# finds weights that give least-square error
# matrix equation: A*weights ~ data
weights = np.matmul(np.linalg.pinv(A),resampled_data).T
print(weights.shape)
print(resampled_data.shape)

# normalize the weights
n = len(weights)
for i in range(n):
  weights[i] = weights[i]/np.linalg.norm(weights[i])

# x is the min value of inner product required to identify as pair
x = .99
results = []
macro_pairs = []
for i in range(n):
  for j in range(min(i + 1, n - 1), n):
    if (np.dot(weights[i], weights[j])>x):
      results.append((samples[i], samples[j]))
      # results.append((i, j))
      macro_pairs.append([i,j])
print(results)
print(len(results))
# print(weights)

(88, 5)
(1705, 88)
[('AM', 'CEQP'), ('AM', 'MRO'), ('AM', 'NOV'), ('AM', 'OXY'), ('AM', 'PSX'), ('AM', 'OILY'), ('AM', 'SUN'), ('AR', 'ENBL'), ('AR', 'NOV'), ('AR', 'OAS'), ('AR', 'SUN'), ('APA', 'AROC'), ('APA', 'CPE'), ('APA', 'CVE'), ('APA', 'DEN'), ('APA', 'KOS'), ('APA', 'MMP'), ('APA', 'PBF'), ('APA', 'PAGP'), ('APA', 'SWN'), ('AROC', 'CPE'), ('AROC', 'DEN'), ('AROC', 'PBF'), ('AROC', 'PAGP'), ('BKR', 'CDEV'), ('BKR', 'DRQ'), ('BKR', 'HP'), ('BKR', 'MTDR'), ('BKR', 'NBLX'), ('BKR', 'SLB'), ('BSM', 'CVI'), ('BSM', 'DKL'), ('BSM', 'FANG'), ('BSM', 'ET'), ('BSM', 'DRQ'), ('BSM', 'EOG'), ('BSM', 'HP'), ('BSM', 'KMI'), ('BSM', 'MUR'), ('BSM', 'NESR'), ('BSM', 'REGI'), ('COG', 'CRK'), ('WHD', 'DNOW'), ('WHD', 'HEP'), ('WHD', 'MPLX'), ('CPE', 'DEN'), ('CPE', 'PBF'), ('CPE', 'PAGP'), ('CPE', 'SWN'), ('CVE', 'CQP'), ('CVE', 'CHK'), ('CVE', 'COP'), ('CVE', 'CLR'), ('CVE', 'GEL'), ('CVE', 'HAL'), ('CVE', 'HES'), ('CVE', 'KOS'), ('CVE', 'MMP'), ('CVE', 'OXY'), ('CVE', 'PDCE'), ('CVE', 'PAGP'

In [183]:
def find_cointegrated_pairs(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                pairs.append((keys[i], keys[j],pvalue))
    return score_matrix, pvalue_matrix, pairs

In [184]:
scores, pvalues, pairs, = find_cointegrated_pairs(prices)
best_pairs = [pair for pair in pairs if pair[2]<.001]

KeyboardInterrupt: 

In [ ]:
sns.heatmap(pvalues, xticklabels=samples, yticklabels=samples, cmap='RdYlGn_r' , mask = (pvalues >= 0.98))
plt.show()

In [ ]:
ticker1 = "AROC"# Set ticker1
ticker2 = "CPE"# Set ticker2
S1 = prices[ticker1]
S2 = prices[ticker2]
score, pvalue, _ = coint(S1, S2)
print(pvalue)

ratios = S1 / S2
#ratios.plot(figsize=(15,7))
#plt.axhline(ratios.mean())
#plt.legend(['Price Ratio'])
#plt.show()

In [ ]:
pairs

In [ ]:
for result in results:
    if result[0] 

In [ ]:
trimmed_pairs = [(pair[0], pair[1]) for pair in pairs]
#trimmed_pairs
combo_pairs = []
for result in results:
    if ((result[0],result[1]) in trimmed_pairs) or ((result[1], result[0]) in trimmed_pairs):
        combo_pairs.append(result)
combo_pairs

In [ ]:
split_number = round(2/3 * len(ratios))
print(split_number)
train = ratios[:split_number]
test = ratios[split_number:]

In [ ]:
total_profit = 0

for pair in combo_pairs:
    ticker1 = pair[0]
    ticker2 = pair[1]
    total_profit += trade(prices[ticker1].iloc[split_number:], prices[ticker2].iloc[split_number:], 5, 120)
average_profit = total_profit/len(combo_pairs)
total_profit

In [ ]:
ticker1 = "DKL"# Set ticker1
ticker2 = "MRO"

In [ ]:
# Compute the rolling z-score of the ratio using the definition above
# Hint: Use pandas.Series.rolling on your training set. Set center = false

ratios_mavg5 = ratios.rolling('5d',center = False).mean() 
# print(ratios_mavg5)

ratios_mavg60 = ratios.rolling('60d',center = False).mean() 
# print(ratios_mavg60)

std_60 = ratios.rolling('60d',center = False).std()
# print(std_60)

zscore_60_5 = (ratios_mavg5 - ratios_mavg60)/std_60
print(zscore_60_5)

In [ ]:
# This plots the moving averages you calculated. Notice how longer windows smoothen out the curves.

plt.figure(figsize=(15,7))
plt.plot(train.index, train.values)
plt.plot(ratios_mavg5.index, ratios_mavg5.values)
plt.plot(ratios_mavg60.index, ratios_mavg60.values)

plt.legend(['Ratio','5d Ratio MA', '60d Ratio MA'])

plt.ylabel('Ratio')
plt.show()

In [ ]:
# This plot the rolling z-score of the ratio

plt.figure(figsize=(15,7))
zscore_60_5.plot()
plt.axhline(0, color='black')
plt.axhline(1.0, color='red', linestyle='--')
plt.axhline(-1.0, color='green', linestyle='--')
plt.legend(['Rolling Ratio z-Score', 'Mean', '+1', '-1'])
plt.show()

In [ ]:
#Generate signals using the previously defined trading strategy

plt.figure(figsize=(15,7))
train[60:].plot()

buy = train.copy()
sell = train.copy()
buy[zscore_60_5>-1] = 0 
sell[zscore_60_5<1] = 0

buy[60:].plot(color='g', linestyle='None', marker='^')
sell[60:].plot(color='r', linestyle='None', marker='^')
x1,x2,y1,y2 = plt.axis()
plt.axis((x1,x2,train.min(),train.max()))
plt.legend(['Ratio', 'Buy Signal', 'Sell Signal'])
plt.show()

In [ ]:
plt.figure(figsize=(18,9))
S1 = prices[ticker1].iloc[:671]
S2 = prices[ticker2].iloc[:671]

S1[60:].plot(color='b')
S2[60:].plot(color='c')
buyR = 0*S1.copy()
sellR = 0*S1.copy()

# When buying the ratio, buy S1 and sell S2
buyR[buy!=0] = S1[buy!=0]
sellR[buy!=0] = S2[buy!=0]

# When selling the ratio, sell S1 and buy S2 
buyR[sell!=0] = S2[sell!=0]
sellR[sell!=0] = S1[sell!=0]

buyR[60:].plot(color='g', linestyle='None', marker='^')
sellR[60:].plot(color='r', linestyle='None', marker='^')
x1,x2,y1,y2 = plt.axis()
plt.axis((x1,x2,min(S1.min(),S2.min()),max(S1.max(),S2.max())))

plt.legend([ticker1,ticker2, 'Buy Signal', 'Sell Signal'])
plt.show()

In [ ]:
def trade(S1, S2, window1, window2):
    # If window length is 0, algorithm doesn't make sense, so exit
    if (window1 == 0) or (window2 == 0):
        return 0
    # Compute rolling mean and rolling standard deviation
    ratios = S1/S2
    ma1 = ratios.rolling(window=window1,
                               center=False).mean()
    ma2 = ratios.rolling(window=window2,
                               center=False).mean()
    std = ratios.rolling(window=window2,
                        center=False).std()
    zscore = (ma1 - ma2)/std
    # Simulate trading
    # Start with no money and no positions
    money = 0
    countS1 = 0
    countS2 = 0
    for i in range(len(ratios)):
        # Sell short if the z-score is > 1
        if zscore[i] > 1:
            money += S1[i] - S2[i] * ratios[i]
            countS1 -= 1
            countS2 += ratios[i]
        # Buy long if the z-score is < 1
        elif zscore[i] < -1:
            money -= S1[i] - S2[i] * ratios[i]
            countS1 += 1
            countS2 -= ratios[i]
        # Clear positions if the z-score between -.5 and .5
        elif abs(zscore[i]) < 0.5:
            money += countS1 * S1[i] + S2[i] * countS2
            countS1 = 0
            countS2 = 0
    return money

In [ ]:
trade(prices[ticker1].iloc[:split_number], prices[ticker2].iloc[:split_number], 5, 60)

In [ ]:
trade(prices[ticker1].iloc[split_number:], prices[ticker2].iloc[split_number:], 5, 60)

In [ ]:
# Check with training data
length_scores = [trade(prices[ticker1].iloc[:split_number], 
                prices[ticker2].iloc[:split_number], 5, l) 
                for l in range(255)]
best_length = np.argmax(length_scores)
print ('Best window length:', best_length)
print('Money made:', length_scores[best_length])

In [ ]:
# Check with testing data
length_scores2 = [trade(prices[ticker1].iloc[split_number:], 
                  prices[ticker2].iloc[split_number:],5, l) 
                  for l in range(255)]

best_length2 = np.argmax(length_scores2)
print ('Best window length:', best_length2)
print('Money made:', length_scores2[best_length2])